In [ ]:
!pip install alpaca-trade-api pandas numpy schedule
!pip install praw pandas transformers torch numpy google-play-scraper
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 732.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [ ]:
from alpaca_trade_api.rest import REST
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
import praw
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from google_play_scraper import reviews

Sentiment Algorithm

In [ ]:
#Scrape Reddit 1
def scrape_reddit(client_id, client_secret, user_agent, query="Chipotle", limit=1000):
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent
    )
    posts = []
    for submission in reddit.subreddit("all").search(query, limit=limit):
        if not submission.stickied:
            posts.append(submission.title + " " + submission.selftext)
    return pd.DataFrame({'text': posts})

#Scrape Google Play reviews 2
def scrape_google_play(app_id="com.chipotle.ordering", limit=200):
    review_data, _ = reviews(
        app_id,
        lang='en',
        country='us',
        count=limit
    )
    return pd.DataFrame({'text': [r['content'] for r in review_data]})

#Load sentiment model HAVE TO CHANGE THIS
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    top_k=None
)

#Sentiment analysis
def analyze_sentiment(text):
    try:
        result = sentiment_pipeline(text)[0]
        scores = {label_map[entry['label']]: entry['score'] for entry in result}
        return max(scores, key=scores.get)
    except Exception as e:
        print(f"Error: {e}")
        return "neutral"


client_id = "M8D3he3NjOj8Y4Z3PF3gzw"
client_secret = "77fOIkC8gN_SC5g_-2TDXKKZZFENXg"
user_agent = "chipotle-analysis by u/No-Chemistry-203"

#Scrape data
print("Scraping Reddit...")
reddit_df = scrape_reddit(client_id, client_secret, user_agent, query="Chipotle", limit=1000)

print("Scraping Google Play...")
google_df = scrape_google_play(app_id="com.chipotle.ordering", limit=200)

#Combine and analyze
combined_df = pd.concat([reddit_df, google_df], ignore_index=True)
combined_df['sentiment'] = combined_df['text'].apply(analyze_sentiment)

#Export
#output_path = r"C:\Users\dpods\Downloads\chipotle_2sentiment.csv"
#combined_df.to_csv(output_path, index=False, encoding='utf-8')
#print(f"Analysis complete, File saved to: {output_path}")

#Check
print("Reddit rows:", len(reddit_df))
print("Google Play rows:", len(google_df))
print("Total combined rows:", len(combined_df))

summary = combined_df['sentiment'].value_counts()
print(summary)

sentiment_counts = combined_df['sentiment'].value_counts()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

Device set to use cpu
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Scraping Reddit...


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraping Google Play...
Error: The expanded size of the tensor (636) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 636].  Tensor sizes: [1, 514]
Error: The expanded size of the tensor (670) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 670].  Tensor sizes: [1, 514]
Error: The expanded size of the tensor (1006) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1006].  Tensor sizes: [1, 514]
Error: The expanded size of the tensor (880) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 880].  Tensor sizes: [1, 514]
Error: The expanded size of the tensor (1097) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1097].  Tensor sizes: [1, 514]
Error: The expanded size of the tensor (649) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 649].  Tensor sizes: [1, 514]
Error: The expande

Uptrend Algorithm

In [ ]:
# variables
API_KEY = "PK0CBK5ROYJZAAEBUWKB"
SECRET_KEY = "Tip93sYY2IAz6Mxgx99A3qBzvlXlVrU3grqhhd9P"
BASE_URL = "https://paper-api.alpaca.markets"
STOCK_SYMBOL = "CMG"
TRADE_AMOUNT = 100

In [ ]:
# connect to API
api = REST(API_KEY, SECRET_KEY, base_url=BASE_URL)

In [ ]:
# fetch 50-day MA function
def get_price_data(symbol):
    end_date = datetime.utcnow() - timedelta(days=1)
    start_date = end_date - timedelta(days=90)

    bars = api.get_bars(
        symbol,
        timeframe="1Day",
        start=start_date.strftime("%Y-%m-%dT%H:%M:%SZ"),
        end=end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    ).df

    df = bars.copy()
    df['MA_50'] = df['close'].rolling(window=50).mean()

    return df


In [ ]:
# this function checks if the most recent closing price is greater than the 50-day MA
def is_uptrend(df):
    latest_close = df['close'].iloc[-1]
    latest_ma = df['MA_50'].iloc[-1]
    return latest_close > latest_ma, latest_close, latest_ma

In [ ]:
# place buy function
def place_buy(symbol, current_price):
    qty = round(TRADE_AMOUNT / current_price, 4)
    order = api.submit_order(
        symbol=symbol,
        qty=qty,
        side="buy",
        type="market",
        time_in_force="day"
    )
    return qty


In [ ]:
# sell function

def place_sell(symbol, qty):
    try:
        order = api.submit_order(
            symbol=symbol,
            qty=qty,
            side='sell',
            type='market',
            time_in_force='day'
        )
        print(f"Sell order placed for {qty} shares of {symbol}")
        return order
    except Exception as e:
        print(f"Error placing sell order for {symbol}: {e}")
        return None



In [ ]:
# csv log
def log_trade(symbol, qty, price, ma_50):
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "symbol": symbol,
        "price": price,
        "ma_50": ma_50,
        "qty": qty,
        "trade_value": round(qty * price, 2)
    }

    log_file = "trade_log.csv"
    if os.path.exists(log_file):
        df = pd.read_csv(log_file)
        df = pd.concat([df, pd.DataFrame([log_entry])], ignore_index=True)
    else:
        df = pd.DataFrame([log_entry])

    df.to_csv(log_file, index=False)
    print(f"Trade logged for {symbol}")


In [ ]:
account = api.get_account()

print("API Connected")
print(f"Account status: {account.status}")
print(f"Equity: ${account.equity}")
print(f"Buying power: ${account.buying_power}")

API Connected
Account status: ACTIVE
Equity: $100000
Buying power: $199499.4


**Notes:**
- make list of stocks
- have run_bot loop through until it finds one with uptrend.
- If it does, buy and move on to sell loop

In [ ]:
# main
positions = {} # check positions

def run_bot(symbol):
    df = get_price_data(symbol)

    if df.empty or df['MA_50'].isna().all():
        print(f"No valid data for {symbol}. Possibly rate-limited or stock symbol is invalid.")
        return

    uptrend, price, ma = is_uptrend(df)

    if symbol in positions:
      entry_price = positions[symbol]['entry_price']
      qty = positions[symbol]['qty']
      change_pct = (price - entry_price) / entry_price

        # if change_pct >= 0.02 # this is our take profit exit
          # print(f"SELL {symbol} + 2%. Entry {en}")

    if uptrend:
        print(f"{symbol} is in uptrend. Price: {price:.2f}, MA50: {ma:.2f}")
        qty = place_trade(symbol, price)
        log_trade(symbol, qty, price, ma)
    else:
        print(f"{symbol} is NOT in uptrend. Price: {price:.2f}, MA50: {ma:.2f}")


In [ ]:
run_bot(STOCK_SYMBOL)

CMG is in uptrend. Price: 50.52, MA50: 50.25
Trade logged for CMG
